In [16]:
import json
import pandas as pd
import glob
import os
import math
import numpy as np

# ==========================================
# [1] 설정값 (Configuration)
# ==========================================
LOG_FOLDER_PATH = 'G:/내 드라이브/pubg_telemetry_summary'
BATCH_SIZE = 100
OUTPUT_FILE_NAME = 'pubg_training_dataset.csv' # 최종 결과 파일명
  
TIME_INTERVAL_SEC = 10      # 스냅샷 주기
START_ALIVE_THRESHOLD = 10  # 기록 시작 트리거
AI_TEAM_ID_START = 200      
MAX_AI_RATIO = 0.4     
     

# [Fix] 거리 단위 수정 (m -> cm)
# PUBG 좌표는 cm 단위이므로 300m = 30,000cm
DANGER_RADIUS_CM = 30000    
STALE_THRESHOLD_SEC = 30    


In [17]:

# ==========================================
# [2] 헬퍼 함수
# ==========================================
def get_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def get_item_level(item_id):
    """아이템 ID에서 레벨(1,2,3) 추출"""
    # 안전한 문자열 처리
    if not isinstance(item_id, str): return 0
    
    if 'Head' in item_id:
        if 'Lv1' in item_id: return 1
        elif 'Lv2' in item_id: return 2
        elif 'Lv3' in item_id: return 3
    if 'Armor' in item_id or 'Vest' in item_id:
        if 'Lv1' in item_id: return 1
        elif 'Lv2' in item_id: return 2
        elif 'Lv3' in item_id: return 3
    return 0

def get_throwable_type(item_id):
    """투척류 종류 판별"""
    if not isinstance(item_id, str): return None
    if 'Smoke' in item_id: return 'smoke'
    if 'Grenade' in item_id and 'Bluezone' not in item_id: return 'grenade' # 수류탄
    if 'Molotov' in item_id: return 'molotov'
    if 'Flash' in item_id: return 'flash'
    return None

def calculate_angle(my_pos, enemy1_pos, enemy2_pos):
    """내 위치 기준 두 적 사이의 각도 계산 (양각 판별)"""
    v1_x = enemy1_pos[0] - my_pos[0]; v1_y = enemy1_pos[1] - my_pos[1]
    v2_x = enemy2_pos[0] - my_pos[0]; v2_y = enemy2_pos[1] - my_pos[1]
    
    norm1 = math.sqrt(v1_x**2 + v1_y**2)
    norm2 = math.sqrt(v2_x**2 + v2_y**2)
    
    if norm1 == 0 or norm2 == 0: return 0
    
    dot_product = (v1_x * v2_x) + (v1_y * v2_y)
    cos_theta = max(-1.0, min(1.0, dot_product / (norm1 * norm2)))
    
    return math.degrees(math.acos(cos_theta))


In [18]:

# ==========================================
# [3] 메인 처리 로직
# ==========================================
def process_match_timeseries(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            logs = [json.loads(line) for line in f]
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None

    # --- 1. AI 비율 필터링 ---
    all_players = set(); human_players = set()
    for log in logs:
        if log.get('_T') == 'LogPlayerPosition':
            char = log.get('character', {})
            t_id = char.get('teamId', -1)
            name = char.get('name')
            if name:
                all_players.add(name)
                if t_id < AI_TEAM_ID_START: human_players.add(name)
    
    total_cnt = len(all_players)
    if total_cnt == 0: return None
    if (1.0 - len(human_players)/total_cnt) > MAX_AI_RATIO: return None

    # --- 2. 변수 초기화 ---
    current_alive_players = 100 
    team_players = {}           
    team_final_ranks = {}       
    
    player_status = {}          
    player_gear = {}            
    player_throwables = {}      
    team_kills = {}             
    
    current_safe_zone = {'x': 400000, 'y': 400000}
    
    next_snapshot_time = 0
    recording_started = False
    target_teams = set()
    
    timeseries_data = []

    # 팀 정보 빌드
    for log in logs:
        if log.get('_T') == 'LogPlayerPosition':
            char = log.get('character', {})
            t_id = char.get('teamId', -1)
            if t_id != -1 and t_id < AI_TEAM_ID_START:
                if t_id not in team_players: team_players[t_id] = set()
                team_players[t_id].add(char.get('name'))
                if char.get('ranking', 0) != 0:
                    team_final_ranks[t_id] = char.get('ranking')

    # --- 3. 로그 순차 처리 ---
    for log in logs:
        log_type = log.get('_T')
        elapsed_time = log.get('elapsedTime', 0)
        
        # (A) 상태 업데이트
        if log_type == 'LogGameStatePeriodic':
            sz = log.get('gameState', {}).get('safetyZonePosition', {})
            if sz: current_safe_zone = {'x': sz.get('x'), 'y': sz.get('y')}

        elif log_type == 'LogPlayerPosition':
            if 'numAlivePlayers' in log:
                current_alive_players = log['numAlivePlayers']
            
            char = log.get('character', {})
            name = char.get('name')
            if name:
                player_status[name] = {
                    'health': char.get('health', 0),
                    'isInVehicle': char.get('isInVehicle', False),
                    'location': char.get('location', {}),
                    'last_update': elapsed_time
                }

        # [Fix 1] 장비는 '착용(Equip)' 로그로 추적 (정확도 향상)
        elif log_type == 'LogItemEquip':
            item = log.get('item', {})
            name = log.get('character', {}).get('name')
            item_id = item.get('itemId', '')
            if name:
                lvl = get_item_level(item_id)
                if lvl > 0:
                    if name not in player_gear: player_gear[name] = {'head':0, 'armor':0}
                    # 해당 부위 장비 덮어쓰기 (교체 반영)
                    if 'Head' in item_id: player_gear[name]['head'] = lvl
                    else: player_gear[name]['armor'] = lvl
        
        # [Fix 1] 투척류는 '획득(Pickup)' 로그로 추적 (소모품이므로 누적)
        elif log_type == 'LogItemPickup':
            item = log.get('item', {})
            name = log.get('character', {}).get('name')
            item_id = item.get('itemId', '')
            if name:
                # 투척류만 처리
                t_type = get_throwable_type(item_id)
                if t_type:
                    stack = item.get('stackCount', 1)
                    if name not in player_throwables:
                        player_throwables[name] = {'smoke':0, 'grenade':0, 'molotov':0, 'flash':0}
                    player_throwables[name][t_type] += stack

        elif log_type == 'LogPlayerKillV2':
            killer = log.get('killer')
            victim = log.get('victim')
            
            if victim:
                v_name = victim.get('name')
                if v_name and v_name in player_status:
                    player_status[v_name]['health'] = 0 
            
            if killer and victim:
                k_tid = killer.get('teamId', -1)
                v_tid = victim.get('teamId', -1)
                if k_tid != -1 and k_tid < AI_TEAM_ID_START:
                    if v_tid != -1 and v_tid < AI_TEAM_ID_START:
                        team_kills[k_tid] = team_kills.get(k_tid, 0) + 1

        # (B) 기록 시작 트리거
        if not recording_started:
            if elapsed_time > 300 and current_alive_players <= START_ALIVE_THRESHOLD:
                recording_started = True
                next_snapshot_time = elapsed_time
                for t_id, members in team_players.items():
                    is_alive = False
                    for m in members:
                        status = player_status.get(m, {})
                        if status.get('health', 0) > 0 and (elapsed_time - status.get('last_update', 0) <= STALE_THRESHOLD_SEC):
                            is_alive = True
                            break
                    if is_alive:
                        target_teams.add(t_id)

        # (C) 스냅샷 기록
        if recording_started and elapsed_time >= next_snapshot_time:
            
            active_team_centroids = {}  
            temp_team_stats = {}        

            # 1. 팀 스탯 계산
            for t_id in target_teams:
                members = team_players[t_id]
                
                alive_cnt=0; hp_sum=0; vehicle_cnt=0
                cnt_lv3=0; cnt_lv2=0
                cnt_smoke=0; cnt_grenade=0
                alive_x=[]; alive_y=[]; alive_z=[]
                
                for member in members:
                    status = player_status.get(member)
                    if status and status.get('health', 0) > 0:
                        if (elapsed_time - status.get('last_update', 0) <= STALE_THRESHOLD_SEC):
                            alive_cnt += 1
                            hp_sum += status['health']
                            if status['isInVehicle']: vehicle_cnt += 1
                            
                            gear = player_gear.get(member, {'head':0, 'armor':0})
                            for g in ['head', 'armor']:
                                if gear[g] == 3: cnt_lv3 += 1
                                elif gear[g] == 2: cnt_lv2 += 1
                            
                            throws = player_throwables.get(member, {'smoke':0, 'grenade':0})
                            cnt_smoke += throws['smoke']
                            cnt_grenade += throws['grenade']

                            loc = status['location']
                            if loc:
                                alive_x.append(loc.get('x',0)); alive_y.append(loc.get('y',0)); alive_z.append(loc.get('z',0))
                        else:
                            player_status[member]['health'] = 0

                dist_to_safe = 999999; avg_height = 0
                if alive_cnt > 0:
                    cx = np.mean(alive_x); cy = np.mean(alive_y)
                    dist_to_safe = get_distance(cx, cy, current_safe_zone['x'], current_safe_zone['y'])
                    # team_spread 제거됨
                    avg_height = np.mean(alive_z)
                    active_team_centroids[t_id] = (cx, cy)
                
                temp_team_stats[t_id] = {
                    'num_alive': alive_cnt,
                    'total_health': hp_sum,
                    'num_lv3_gear': cnt_lv3,
                    'num_lv2_gear': cnt_lv2,
                    'num_smokes': cnt_smoke,
                    'num_grenades': cnt_grenade,
                    'has_vehicle': 1 if vehicle_cnt > 0 else 0,
                    'dist_to_safezone': dist_to_safe,
                    'avg_height': avg_height,
                    'kills': team_kills.get(t_id, 0)
                }

            # 2. 적 거리 및 양각(Angle) 계산 [Fix 2: 단위 수정]
            for t_id, stats in temp_team_stats.items():
                nearest_dist = 999999; max_angle = 0
                
                if stats['num_alive'] > 0 and t_id in active_team_centroids:
                    my_pos = active_team_centroids[t_id]
                    nearby_enemies = []
                    
                    for enemy_id, enemy_pos in active_team_centroids.items():
                        if t_id == enemy_id: continue
                        dist = get_distance(my_pos[0], my_pos[1], enemy_pos[0], enemy_pos[1])
                        if dist < nearest_dist: nearest_dist = dist
                        
                        # [Fix 2] cm 단위 비교 (30000 = 300m)
                        if dist <= DANGER_RADIUS_CM:
                            nearby_enemies.append(enemy_pos)
                    
                    if len(nearby_enemies) >= 2:
                        for i in range(len(nearby_enemies)):
                            for j in range(i+1, len(nearby_enemies)):
                                angle = calculate_angle(my_pos, nearby_enemies[i], nearby_enemies[j])
                                if angle > max_angle: max_angle = angle
                
                if nearest_dist == 999999: nearest_dist = -1
                
                final_rank = team_final_ranks.get(t_id, 100)
                is_winner = 1 if final_rank == 1 else 0
                
                timeseries_data.append({
                    'match_id': os.path.basename(file_path),
                    'time_sec': next_snapshot_time,
                    'team_id': t_id,
                    'num_alive': stats['num_alive'],
                    'total_health': stats['total_health'],
                    'num_lv3_gear': stats['num_lv3_gear'],
                    'num_lv2_gear': stats['num_lv2_gear'],
                    'num_smokes': stats['num_smokes'],
                    'num_grenades': stats['num_grenades'],
                    'kills': stats['kills'],
                    'has_vehicle': stats['has_vehicle'],
                    'dist_to_safezone': stats['dist_to_safezone'],
                    'dist_to_nearest_enemy': nearest_dist,
                    'max_enemy_angle': max_angle,
                    'avg_height': stats['avg_height'],
                    'is_winner': is_winner
                })
            
            next_snapshot_time += TIME_INTERVAL_SEC

    return pd.DataFrame(timeseries_data) if timeseries_data else None


In [ ]:
import time
# ==========================================
# [4] 실행부 (Batch Processor with Safety)
# ==========================================
if __name__ == "__main__":
    # 1. 파일 목록 로드
    patterns = ["*squad_Baltic_Main.jsonl", "*squad_Tiger_Main.jsonl", "*squad_Desert_Main.jsonl"]
    all_files = []
    
    for pattern in patterns:
        all_files.extend(glob.glob(os.path.join(LOG_FOLDER_PATH, pattern)))
        
    total_files = len(all_files)
    print(f"📂 폴더 탐색 완료: {total_files}개 파일 발견 (Baltic & Tiger)")
    print(f"🚀 분석 시작 (배치 크기: {BATCH_SIZE}개)")
    
    start_time = time.time()
    df_buffer = []
    processed_count = 0
    error_count = 0
    skipped_count = 0
    
    # [Safety] 기존 파일 있으면 삭제 (덮어쓰기)
    if os.path.exists(OUTPUT_FILE_NAME):
        os.remove(OUTPUT_FILE_NAME)
        print(f"ℹ️ 기존 {OUTPUT_FILE_NAME} 삭제됨 (새로 생성)")

    # 2. 파일 순회 및 처리
    for i, fpath in enumerate(all_files):
        try:
            res = process_match_timeseries(fpath)
            
            if res is not None:
                df_buffer.append(res)
                processed_count += 1
            else:
                skipped_count += 1
                
        except Exception as e:
            print(f"⚠️ Error in {os.path.basename(fpath)}: {e}")
            error_count += 1
        
        # 3. [Safety] 배치 단위 저장 (메모리 비우기)
        if (i + 1) % BATCH_SIZE == 0 or (i + 1) == total_files:
            if df_buffer:
                batch_df = pd.concat(df_buffer, ignore_index=True)
                
                # 첫 배치면 헤더 포함 저장, 이후는 헤더 없이 이어쓰기(append)
                if not os.path.exists(OUTPUT_FILE_NAME):
                    batch_df.to_csv(OUTPUT_FILE_NAME, index=False, mode='w', encoding='utf-8-sig')
                else:
                    batch_df.to_csv(OUTPUT_FILE_NAME, index=False, mode='a', header=False, encoding='utf-8-sig')
                
                print(f"💾 [Auto Save] {len(batch_df)}행 저장 완료 (진행률: {i+1}/{total_files})")
                df_buffer.clear() # 메모리 해제
            else:
                print(f"ℹ️ [Auto Save] 저장할 데이터가 없어 건너뜀 (진행률: {i+1}/{total_files})")

    elapsed_time = time.time() - start_time
    print("\n" + "="*30)
    print(f"✅ 모든 작업 완료! ({elapsed_time:.1f}초 소요)")
    print(f" - 성공: {processed_count}개")
    print(f" - 스킵(AI/데이터부족): {skipped_count}개")
    print(f" - 에러: {error_count}개")
    print(f"📂 최종 파일: {OUTPUT_FILE_NAME}")
    print("="*30)

📂 폴더 탐색 완료: 3848개 파일 발견 (Baltic & Tiger)
🚀 분석 시작 (배치 크기: 100개)
ℹ️ 기존 pubg_training_dataset.csv 삭제됨 (새로 생성)
💾 [Auto Save] 2944행 저장 완료 (진행률: 100/3848)
💾 [Auto Save] 5141행 저장 완료 (진행률: 200/3848)
